Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [6]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [7]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

Modified the Graph function to accept either SGD or NN logits 

In [1]:
def SGD_logits(train, weights, biases):
    logits = tf.matmul(train, weights[0]) + biases[0]
    return logits


def NN_logits(train, weights, biases):

    for i in range(len(weights)):
        logits = tf.matmul(train, weights[i]) + biases[i]
        train = tf.nn.relu(logits)
    return logits

These functions will handle logit computation for the graph

dimension_list is a variable that lists out the dimensions of the hidden nodes
       X =>   hidden_1 => hidden_2 => ... => num_labels
       
generate_variables is a function defined in the graph to make the weighs and the biases according to the dimensions specified in dimension_list

In [2]:
batch_size = 128
hidden_nodes = 1024
num_steps = 3001

"""
dimension_list = [image_size*image_size, num_labels]
logit_maker = SGD_logits
"""


dimension_list = [image_size*image_size, hidden_nodes, num_labels]
logit_maker = NN_logits

##################
# MODIFIED GRAPH
####################
graph = tf.Graph()
with graph.as_default():
    def generate_variables(dimension_list):
        weights = []
        biases = []
        for i in range(1, len(dimension_list)):
            weights.append(tf.Variable(tf.truncated_normal(
                [dimension_list[i-1], dimension_list[i]])))
            biases.append(tf.Variable(tf.zeros([dimension_list[i]])))
        return weights, biases

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta = tf.placeholder(tf.float32)

    # Variables.
    weights, biases = generate_variables(dimension_list)
    # Training computation.
    logits = logit_maker(tf_train_dataset,  weights, biases)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    for weight in weights:
        loss = loss + beta*tf.nn.l2_loss(weight)

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_logits = logit_maker(tf_valid_dataset, weights, biases)
    valid_prediction = tf.nn.softmax(valid_logits)
    test_logits = logit_maker(tf_test_dataset, weights, biases)
    test_prediction = tf.nn.softmax(test_logits)




NameError: name 'image_size' is not defined

Session left unchanged except placed inside a for loop to search out best beta value 
<br>
for the 1_layer_NN 
<br>
best_beta = 0.001584893192461114
<br>
max_acc = 93.1

In [ ]:
beta_vals = [10**i for i in np.linspace(-3, -1, 21)]
test_acc = []
#beta_vals=[0]
for beta_val in beta_vals:
    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        print("Initialized")
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset: batch_data,
                         tf_train_labels: batch_labels, beta: beta_val}
            _, l, predictions = session.run(
                [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % nmm.accuracy(predictions, batch_labels))
                print("Validation accuracy: %.1f%%" % nmm.accuracy(
                    valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%" % nmm.accuracy(test_prediction.eval(), test_labels))
        test_acc.append(nmm.accuracy(test_prediction.eval(), test_labels))
        
plt.semilogx(beta_vals, test_acc)

index = np.argmax(np.array(test_acc))
best_beta = beta_vals[index]
best_beta
max(test_acc)


Results at best_beta

Initialized
Minibatch loss at step 0: 700.051819
Minibatch accuracy: 8.6%
Validation accuracy: 41.3%
Minibatch loss at step 500: 217.382782
Minibatch accuracy: 82.8%
Validation accuracy: 79.8%
Minibatch loss at step 1000: 111.139748
Minibatch accuracy: 78.9%
Validation accuracy: 81.2%
Minibatch loss at step 1500: 58.254711
Minibatch accuracy: 91.4%
Validation accuracy: 83.8%
Minibatch loss at step 2000: 31.059258
Minibatch accuracy: 90.6%
Validation accuracy: 85.8%
Minibatch loss at step 2500: 16.789158
Minibatch accuracy: 88.3%
Validation accuracy: 86.2%
Minibatch loss at step 3000: 9.224454
Minibatch accuracy: 85.9%
Validation accuracy: 87.0%
Test accuracy: 93.1%

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [ ]:
best_beta =  0.001584893192461114 

num_batches=3

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        offset = (step % num_batches* batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset: batch_data,
                     tf_train_labels: batch_labels, beta: best_beta}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % nmm.accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % nmm.accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % nmm.accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 773.293823
Minibatch accuracy: 10.2%
Validation accuracy: 38.6%
Minibatch loss at step 500: 210.755997
Minibatch accuracy: 100.0%
Validation accuracy: 75.5%
Minibatch loss at step 1000: 112.285080
Minibatch accuracy: 100.0%
Validation accuracy: 75.5%
Minibatch loss at step 1500: 59.822441
Minibatch accuracy: 100.0%
Validation accuracy: 75.6%
Minibatch loss at step 2000: 31.871960
Minibatch accuracy: 100.0%
Validation accuracy: 75.5%
Minibatch loss at step 2500: 16.981649
Minibatch accuracy: 100.0%
Validation accuracy: 75.7%
Minibatch loss at step 3000: 9.052319
Minibatch accuracy: 100.0%
Validation accuracy: 76.1%
Test accuracy: 83.8%

Minibatch accuracy quickly reaches 100 while validation and test accuracy plateau

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [ ]:
def NN_logits(train, weights, biases, prob=1.0):

    for i in range(len(weights)):
        logits = tf.matmul(train, weights[i]) + biases[i]
        train = tf.nn.relu(logits)
        train = tf.nn.dropout(train, prob)
    return logits

logit_maker = NN_logits

graph = tf.Graph()
with graph.as_default():
    def generate_variables(dimension_list):
        weights = []
        biases = []
        for i in range(1, len(dimension_list)):
            weights.append(tf.Variable(tf.truncated_normal(
                [dimension_list[i-1], dimension_list[i]])))
            biases.append(tf.Variable(tf.zeros([dimension_list[i]])))
        return weights, biases

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta = tf.placeholder(tf.float32)

    # Variables.
    weights, biases = generate_variables(dimension_list)
    # Training computation.
    logits = logit_maker(tf_train_dataset,  weights, biases, 0.5)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))

    for weight in weights:
        loss = loss + beta*tf.nn.l2_loss(weight)

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_logits = logit_maker(tf_valid_dataset, weights, biases)
    valid_prediction = tf.nn.softmax(valid_logits)
    test_logits = logit_maker(tf_test_dataset, weights, biases)
    test_prediction = tf.nn.softmax(test_logits)

Rerun session from before

Initialized
Minibatch loss at step 0: 945.241516
Minibatch accuracy: 9.4%
Validation accuracy: 25.4%
Minibatch loss at step 500: 211.780579
Minibatch accuracy: 100.0%
Validation accuracy: 76.5%
Minibatch loss at step 1000: 112.868469
Minibatch accuracy: 100.0%
Validation accuracy: 76.4%
Minibatch loss at step 1500: 60.146503
Minibatch accuracy: 100.0%
Validation accuracy: 76.7%
Minibatch loss at step 2000: 32.046276
Minibatch accuracy: 100.0%
Validation accuracy: 76.9%
Minibatch loss at step 2500: 17.074589
Minibatch accuracy: 100.0%
Validation accuracy: 76.6%
Minibatch loss at step 3000: 9.099283
Minibatch accuracy: 100.0%
Validation accuracy: 76.7%
Test accuracy: 84.3%

slight increase in test accuracy

test accuracy goes down small amount when using not overfit graph

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [ ]:
dimension_list = [image_size*image_size, hidden_nodes, hidden_nodes//2, hidden_nodes//4, hidden_nodes//8, num_labels]
logit_maker = NN_logits

graph = tf.Graph()
with graph.as_default():
    def generate_variables(dimension_list):
        weights = []
        biases = []
        for i in range(1, len(dimension_list)):
            weights.append(tf.Variable(tf.truncated_normal(
                [dimension_list[i-1], dimension_list[i]], stddev=np.sqrt(2.0/dimension_list[i-1]))))
            biases.append(tf.Variable(tf.zeros([dimension_list[i]])))
        return weights, biases

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta = tf.placeholder(tf.float32)
    global_step = tf.Variable(0)

    # Variables.
    weights, biases = generate_variables(dimension_list)
    # Training computation.
    logits = logit_maker(tf_train_dataset,  weights, biases, 0.5)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))

    for weight in weights:
        loss = loss + beta*tf.nn.l2_loss(weight)

    # Optimizer.
    learning_rate = tf.train.exponential_decay(0.5, global_step, 1000, 0.80, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_logits = logit_maker(tf_valid_dataset, weights, biases)
    valid_prediction = tf.nn.softmax(valid_logits)
    test_logits = logit_maker(tf_test_dataset, weights, biases)
    test_prediction = tf.nn.softmax(test_logits)

num_steps = 20001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset: batch_data,
                     tf_train_labels: batch_labels, beta: 1e-3}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % nmm.accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % nmm.accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % nmm.accuracy(test_prediction.eval(), test_labels))

This 4 layer NN can only achieve 95.8% test validation. There is probably more tweaking that can be done to all the parameters
the beta parameter was just assigned since previous best_beta would no longer work for deeper networks.

Initialized
Minibatch loss at step 0: 4.592533
Minibatch accuracy: 5.5%
Validation accuracy: 11.8%
Minibatch loss at step 500: 1.515888
Minibatch accuracy: 87.5%
Validation accuracy: 83.1%
Minibatch loss at step 1000: 1.401958
Minibatch accuracy: 80.5%
Validation accuracy: 84.2%
Minibatch loss at step 1500: 0.938252
Minibatch accuracy: 88.3%
Validation accuracy: 85.4%
Minibatch loss at step 2000: 0.851192
Minibatch accuracy: 88.3%
Validation accuracy: 86.0%
Minibatch loss at step 2500: 0.823489
Minibatch accuracy: 84.4%
Validation accuracy: 86.2%
Minibatch loss at step 3000: 0.805953
Minibatch accuracy: 85.9%
Validation accuracy: 86.6%
Minibatch loss at step 3500: 0.728941
Minibatch accuracy: 85.9%
Validation accuracy: 87.0%
Minibatch loss at step 4000: 0.672844
Minibatch accuracy: 90.6%
Validation accuracy: 87.2%
Minibatch loss at step 4500: 0.637506
Minibatch accuracy: 85.9%
Validation accuracy: 87.6%
Minibatch loss at step 5000: 0.637102
Minibatch accuracy: 85.9%
Validation accuracy: 87.9%
Minibatch loss at step 5500: 0.760427
Minibatch accuracy: 83.6%
Validation accuracy: 88.0%
Minibatch loss at step 6000: 0.638273
Minibatch accuracy: 86.7%
Validation accuracy: 87.9%
Minibatch loss at step 6500: 0.486741
Minibatch accuracy: 90.6%
Validation accuracy: 88.4%
Minibatch loss at step 7000: 0.639844
Minibatch accuracy: 88.3%
Validation accuracy: 88.5%
Minibatch loss at step 7500: 0.636015
Minibatch accuracy: 84.4%
Validation accuracy: 88.6%
Minibatch loss at step 8000: 0.756223
Minibatch accuracy: 81.2%
Validation accuracy: 88.8%
Minibatch loss at step 8500: 0.521703
Minibatch accuracy: 90.6%
Validation accuracy: 89.2%
Minibatch loss at step 9000: 0.527586
Minibatch accuracy: 89.8%
Validation accuracy: 89.0%
Minibatch loss at step 9500: 0.564700
Minibatch accuracy: 88.3%
Validation accuracy: 89.2%
Minibatch loss at step 10000: 0.519875
Minibatch accuracy: 87.5%
Validation accuracy: 89.2%
Minibatch loss at step 10500: 0.456685
Minibatch accuracy: 89.8%
Validation accuracy: 89.3%
Minibatch loss at step 11000: 0.481208
Minibatch accuracy: 90.6%
Validation accuracy: 89.6%
Minibatch loss at step 11500: 0.513526
Minibatch accuracy: 87.5%
Validation accuracy: 89.7%
Minibatch loss at step 12000: 0.587778
Minibatch accuracy: 86.7%
Validation accuracy: 89.6%
Minibatch loss at step 12500: 0.490071
Minibatch accuracy: 88.3%
Validation accuracy: 89.8%
Minibatch loss at step 13000: 0.623221
Minibatch accuracy: 85.9%
Validation accuracy: 89.8%
Minibatch loss at step 13500: 0.524691
Minibatch accuracy: 88.3%
Validation accuracy: 89.9%
Minibatch loss at step 14000: 0.557879
Minibatch accuracy: 85.2%
Validation accuracy: 90.0%
Minibatch loss at step 14500: 0.625119
Minibatch accuracy: 85.9%
Validation accuracy: 90.0%
Minibatch loss at step 15000: 0.488735
Minibatch accuracy: 89.8%
Validation accuracy: 90.0%
Minibatch loss at step 15500: 0.487046
Minibatch accuracy: 90.6%
Validation accuracy: 90.1%
Minibatch loss at step 16000: 0.387694
Minibatch accuracy: 92.2%
Validation accuracy: 90.0%
Minibatch loss at step 16500: 0.500416
Minibatch accuracy: 91.4%
Validation accuracy: 90.0%
Minibatch loss at step 17000: 0.340107
Minibatch accuracy: 94.5%
Validation accuracy: 90.2%
Minibatch loss at step 17500: 0.274398
Minibatch accuracy: 95.3%
Validation accuracy: 90.2%
Minibatch loss at step 18000: 0.359431
Minibatch accuracy: 93.8%
Validation accuracy: 90.3%
Minibatch loss at step 18500: 0.399571
Minibatch accuracy: 90.6%
Validation accuracy: 90.2%
Minibatch loss at step 19000: 0.344663
Minibatch accuracy: 96.1%
Validation accuracy: 90.2%
Minibatch loss at step 19500: 0.420528
Minibatch accuracy: 91.4%
Validation accuracy: 90.3%
Minibatch loss at step 20000: 0.559027
Minibatch accuracy: 85.2%
Validation accuracy: 90.3%
Test accuracy: 95.8%